#### Load ENV and Model

In [1]:
from dotenv import load_dotenv
load_dotenv()

from langchain.chat_models import init_chat_model

model = init_chat_model(
    model="llama-3.1-8b-instant", 
    model_provider="groq"
)

#### Creating a External Tools

In [2]:
import os
import smtplib
from email.message import EmailMessage
from langchain_core.tools import tool
from dotenv import load_dotenv

load_dotenv()

@tool
def send_real_email(to: str, subject: str, body: str) -> str:
    """Send a real email using Gmail SMTP."""
    sender = os.getenv("EMAIL_USER")
    password = os.getenv("EMAIL_PASSWORD")

    msg = EmailMessage()
    msg["From"] = sender
    msg["To"] = to
    msg["Subject"] = subject
    msg.set_content(body)

    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
        server.login(sender, password)
        server.send_message(msg)

    return "Email sent"


##### Send an Real Email to User

In [3]:
from langchain_core.tools import tool
from langchain_tavily import TavilySearch

tavily = TavilySearch()

@tool
def search_web(query: str) -> str:
    """Search the web using Tavily."""
    results = tavily.invoke(query)

    return str(results)

@tool
def send_email(to: str, subject: str, body: str) -> str:
    """
    Send an email to a recipient.
    """
    return (
        f"Email sent successfully!\n"
        f"To: {to}\n"
        f"Subject: {subject}\n"
        f"Body: {body}"
    )

@tool
def get_weather(city:str) -> str:
    """Get the weather for a city."""
    return f"The weather in {city} is sunny."

d:\LangChainUpdated\.venv\Lib\site-packages\langchain_tavily\tavily_research.py:97: UserWarning: Field name "output_schema" in "TavilyResearch" shadows an attribute in parent "BaseTool"
  class TavilyResearch(BaseTool):  # type: ignore[override, override]
d:\LangChainUpdated\.venv\Lib\site-packages\langchain_tavily\tavily_research.py:97: UserWarning: Field name "stream" in "TavilyResearch" shadows an attribute in parent "BaseTool"
  class TavilyResearch(BaseTool):  # type: ignore[override, override]


#### Create Custom Tools

In [4]:
#### Custom Tools

def add(a:int,b:int)->int:
    """Add two numbers"""
    return a+b

def multiply(a:int,b:int)->int:
    """Multiply two numbers"""
    return a*b

def divide(a:int,b:int)->int:
    """Divide two numbers"""
    return a/b

#### Merge all Tools

In [5]:
tools = [send_real_email,search_web,send_email,get_weather,add,divide,multiply]

#### Creating an Agent

In [6]:
from langchain.agents import create_agent

agent = create_agent(
    model = model,        
    tools=tools,
    system_prompt="You are a helpful research assistant."
)

In [7]:
### Using Add Tool
agent.invoke({
    "messages":[
        {
            "role":"user",
            "content":"what is the output of 2+2?"
        }
    ]
})

{'messages': [HumanMessage(content='what is the output of 2+2?', additional_kwargs={}, response_metadata={}, id='c40833dd-a873-4295-a463-1ab493a48410'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'qmv156m8b', 'function': {'arguments': '{"a":2,"b":2}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 575, 'total_tokens': 593, 'completion_time': 0.0393587, 'completion_tokens_details': None, 'prompt_time': 0.033131365, 'prompt_tokens_details': None, 'queue_time': 0.062493585, 'total_time': 0.072490065}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b7911-877e-7e60-897a-4fe6dfc05baf-0', tool_calls=[{'name': 'add', 'args': {'a': 2, 'b': 2}, 'id': 'qmv156m8b', 'type': 'tool_call'}], usage_metadata={'input_tokens': 575, 'output_tokens': 18, 'total_tokens': 

In [8]:
## Using Multiply tool

multiplication = agent.invoke({
    "messages":[
        {
            "role":"user",
            "content":"what is the output of 100 multiply by 21?",
            
        }
    ]
})
multiplication

{'messages': [HumanMessage(content='what is the output of 100 multiply by 21?', additional_kwargs={}, response_metadata={}, id='845b63e8-34b8-4aed-97fb-be8310487765'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'qy2z2zbm1', 'function': {'arguments': '{"a":100,"b":21}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 577, 'total_tokens': 596, 'completion_time': 0.03829569, 'completion_tokens_details': None, 'prompt_time': 0.033263811, 'prompt_tokens_details': None, 'queue_time': 0.055106359, 'total_time': 0.071559501}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b7911-90e2-7173-af3b-2035998857c0-0', tool_calls=[{'name': 'multiply', 'args': {'a': 100, 'b': 21}, 'id': 'qy2z2zbm1', 'type': 'tool_call'}], usage_metadata={'input_tokens': 577, 'outp

In [9]:
### Using Divide Tool

division = agent.invoke({
    "messages":[
        {
            "role":"user",
            "content":"what is the output of 100 divide by 21?",

        }
    ]
})
division


{'messages': [HumanMessage(content='what is the output of 100 divide by 21?', additional_kwargs={}, response_metadata={}, id='6230c3fb-7149-40d1-8cfb-e8632f00bb97'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '0j6rhkpp8', 'function': {'arguments': '{"a":100,"b":21}', 'name': 'divide'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 577, 'total_tokens': 596, 'completion_time': 0.050306384, 'completion_tokens_details': None, 'prompt_time': 0.033534475, 'prompt_tokens_details': None, 'queue_time': 0.061934494, 'total_time': 0.083840859}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b7911-9c82-7642-9d4b-7b416f6d064c-0', tool_calls=[{'name': 'divide', 'args': {'a': 100, 'b': 21}, 'id': '0j6rhkpp8', 'type': 'tool_call'}], usage_metadata={'input_tokens': 577, 'output_to

In [10]:
real_email = agent.invoke({
    "to": "husensufi213.ac.in",
    "subject": "Interview Update",
    "body": "Congratulations! You are selected."
})
real_email

{'messages': [AIMessage(content="I'm ready to assist you. What would you like to do?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 560, 'total_tokens': 575, 'completion_time': 0.026944691, 'completion_tokens_details': None, 'prompt_time': 0.031850921, 'prompt_tokens_details': None, 'queue_time': 0.056024058, 'total_time': 0.058795612}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b7911-a95c-7c62-a898-1ca0d3eb3461-0', usage_metadata={'input_tokens': 560, 'output_tokens': 15, 'total_tokens': 575})]}

In [11]:
mails = send_real_email.invoke({
    "to": "husensufi213@gmail.com",
    "subject": "Interview Update",
    "body": "Congratulations! You are selected."
})

mails

'Email sent'